In [4]:
import yaml
import torch
import numpy as np
from tqdm import tqdm
from model.sem import SeqEnsembleModel
from bpr.model.bpr_rec import SeqLearn
from torch.utils.data import DataLoader
from data import Data

In [ ]:
with open("config/bpr.yaml", 'r', encoding='utf-8') as f:
    args = yaml.unsafe_load(f)
args

In [ ]:
data = Data(args['data'])
train_loader = DataLoader(data.train_dataset, batch_size=args['batch_size'], shuffle=True)

In [18]:
test_loader = DataLoader(data.test_dataset, batch_size=1, shuffle=False)

In [19]:
user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds = next(iter(test_loader))

In [ ]:
user_ids

In [ ]:
all_scores = model.predict(user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds)
scores, indices = torch.topk(all_scores, 10)
scores, indices

In [ ]:
sem.item_embeddings = sem.item_embeddings.to(sem.device)
seq_emb = sem.item_embeddings(user_seq)
user_emb = sem.user_embeddings(user_ids)  # [batch_size, hidden_dim]

# 添加位置编码
positions = torch.arange(sem.seq_max_len, device=sem.device).expand(user_seq.size(0), -1)
seq_emb = seq_emb + sem.pos_embedding(positions)

# 创建注意力掩码
mask = (user_seq == -1)
output = sem.user_encoder(seq_emb.transpose(0,1), src_key_padding_mask=mask).transpose(0,1)
preference = output[:,-1,:] + user_emb

base_model_emb = sem.item_embeddings(base_model_preds)  # [batch_size, n_base_model, seq_len, hidden_dim]

# 时间衰减权重
time_weights = 1.0 / torch.log2(torch.arange(sem.seq_max_len, device=sem.device) + 2)
time_weights = time_weights.view(1, 1, -1, 1)

basemodel_emb = sem.base_model_embeddings + torch.sum(time_weights * base_model_emb, dim=2)

# 计算基模型权重
wgts_org = torch.sum(preference.unsqueeze(1) * basemodel_emb, dim=-1)  # [batch_size, n_base_model]
import torch.nn.functional as F
wgts = F.softmax(wgts_org, dim=-1)

all_scores = torch.sum(wgts.unsqueeze(2) * all_item_scores, dim=1)  # bc

scores, indices = torch.topk(all_scores, 10)
scores, indices

## 改进模型

In [ ]:
model = SeqLearn(args['model'], args['data'], data.n_user, data.n_item)
ckpt = torch.load(f"../bpr/ckpt_score_sum/bpr_epoch9.pth")
filtered_ckpt = {k: v for k, v in ckpt.items() if not k.startswith('item_tower.cex')}
model.load_state_dict(filtered_ckpt, strict=False)
model.eval()

In [ ]:
with torch.no_grad():
    ndcg_scores = []
    for batch in tqdm(test_loader, desc="计算测试集NDCG"):
        user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds = batch

        all_scores = model.predict(user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds)
        scores, indices = torch.topk(all_scores, 20)
        indices += 1

        for i in range(len(user_ids)):
            user_id = user_ids[i].item()
            pos_item = pos_items[i].item()

            true_items = data.user_interacted_items[data.id_to_user[user_id].item()]
            true_items = true_items[true_items.index(pos_items[i]) + 1:]

            predicted_items = np.array([indices[i].cpu().numpy().tolist()])
            ndcg = nDCG(np.array(predicted_items), [true_items])
            ndcg_scores.append(ndcg)

np.mean(ndcg_scores)

## SEM

In [ ]:
sem = SeqEnsembleModel(args['model'], args['data'], data.n_user, 3952)
ckpt = torch.load(f"../bpr/ckpt_sem/sem_epoch3.pth")
sem.load_state_dict(ckpt, strict=False)
sem.eval()

In [ ]:
test_loader = DataLoader(data.test_dataset, batch_size=args['batch_size'], shuffle=False)

with torch.no_grad():
    ndcg_scores = []
    for batch in tqdm(test_loader, desc="计算测试集NDCG"):
        user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds = batch

        sem.item_embeddings = sem.item_embeddings.to(sem.device)
        seq_emb = sem.item_embeddings(user_seq)
        user_emb = sem.user_embeddings(user_ids)  # [batch_size, hidden_dim]

        # 添加位置编码
        positions = torch.arange(sem.seq_max_len, device=sem.device).expand(user_seq.size(0), -1)
        seq_emb = seq_emb + sem.pos_embedding(positions)

        # 创建注意力掩码
        mask = (user_seq == -1)
        output = sem.user_encoder(seq_emb.transpose(0,1), src_key_padding_mask=mask).transpose(0,1)
        preference = output[:,-1,:] + user_emb

        base_model_emb = sem.item_embeddings(base_model_preds)  # [batch_size, n_base_model, seq_len, hidden_dim]

        # 时间衰减权重
        time_weights = 1.0 / torch.log2(torch.arange(sem.seq_max_len, device=sem.device) + 2)
        time_weights = time_weights.view(1, 1, -1, 1)

        basemodel_emb = sem.base_model_embeddings + torch.sum(time_weights * base_model_emb, dim=2)

        # 计算基模型权重
        wgts_org = torch.sum(preference.unsqueeze(1) * basemodel_emb, dim=-1)  # [batch_size, n_base_model]
        import torch.nn.functional as F
        wgts = F.softmax(wgts_org, dim=-1)

        all_scores = torch.sum(wgts.unsqueeze(2) * all_item_scores, dim=1)  # bc

        # all_scores = model.predict(user_ids, user_seq, all_item_scores, base_model_preds)
        scores, indices = torch.topk(all_scores, 20)
        indices += 1

        for i in range(len(user_ids)):
            user_id = user_ids[i].item()
            pos_item = pos_items[i].item()

            true_items = data.user_interacted_items[data.id_to_user[user_id].item()]
            true_items = true_items[true_items.index(pos_items[i]) + 1:]

            predicted_items = np.array([indices[i].cpu().numpy().tolist()])
            ndcg = nDCG(np.array(predicted_items), [true_items])
            ndcg_scores.append(ndcg)

np.mean(ndcg_scores)

## 基模型预测

In [31]:
acf = np.load(args['data']['base_model_path'] + f"/acf.npy")

In [ ]:
test_loader = DataLoader(data.test_dataset, batch_size=1, shuffle=False)
model = np.load(args['data']['base_model_path'] + f"/sasrec.npy")

ndcg_scores = []
phar = tqdm(test_loader, desc="计算NDCG@10...")
for batch in phar:
    user_ids, user_seq, pos_items, neg_items, all_item_scores, base_model_preds = batch

    user_id = user_ids.item()
    pos_item = pos_items.item()
    interaction_idx = data.get_interaction_index(data.id_to_user[user_id], pos_item)
    assert interaction_idx != -1

    predicted_items = model[interaction_idx][2:2+20]

    predicted_items += 1

    # 获取用户的实际交互物品
    true_items = data.user_interacted_items[user_id]
    true_items = data.user_interacted_items[data.id_to_user[user_id].item()]
    true_items = true_items[true_items.index(pos_item) + 1:]

    ndcg = nDCG(np.array(np.array([predicted_items])), [true_items])
    ndcg_scores.append(ndcg)

    phar.set_postfix(ndcg=ndcg)

np.mean(ndcg_scores)

In [ ]:
all_scores = model.predict(user_ids, user_seq, pos_items, base_model_preds)
_, indices = torch.topk(all_scores, 10)
indices

In [ ]:
scores, indices = torch.topk(all_scores, 10)
scores, indices + 1

In [ ]:
true_items = generator.user_interacted_items[generator.id_to_user[user_ids.item()].item()]
len(true_items), true_items[:5], pos_items

In [ ]:
true_items = generator.user_interacted_items[generator.id_to_user[user_ids.item()].item()]
len(true_items), true_items[:5], pos_items

In [ ]:
true_items_clip = true_items[true_items.index(pos_items.item()) + 1:]
len(true_items_clip), true_items_clip[:10]

In [ ]:
x = torch.tensor(2863).unsqueeze(0).to(model.device)
y = torch.tensor(1).unsqueeze(0).to(model.device)

pos_score, neg_score = model(user_ids, user_seq, pos_items, neg_items, base_model_preds)
pos_score, neg_score

In [ ]:
x = torch.tensor(2863).unsqueeze(0).to(model.device)
y = torch.tensor(1).unsqueeze(0).to(model.device)

pos_score, neg_score = model(user_ids, user_seq, pos_items, neg_items, base_model_preds)
pos_score, neg_score

In [ ]:
my_data = DataLoader(train_dataset, batch_size=1, shuffle=True)
user_ids, user_seq, pos_items, neg_items, base_model_preds = next(iter(my_data))
user_ids, user_seq, pos_items, neg_items

In [ ]:
from data import BPRLoss
loss = BPRLoss()

In [ ]:
import numpy as np
np.random.seed(2021)
 
class Model:
    def __init__(self, k):
        self.k = k
        self.item_size = 50
 
    def __call__(self, users):
        # 模型随机返回 k 个 item,模拟推荐结果
        res = np.random.randint(0, self.item_size, users.shape[0] * self.k)
        return res.reshape((users.shape[0], -1))
 
 
def get_implict_matrix(rec_items, test_set):
    rel_matrix = [[0] * rec_items.shape[1] for _ in range(rec_items.shape[0])]
    for user in range(len(test_set)):
        for index, item in enumerate(rec_items[user]):
            if item in test_set[user]:
                rel_matrix[user][index] = 1
    return np.array(rel_matrix)
 
 
def nDCG(rec_items, test_set):
    DCG = lambda x: np.sum(x / np.log(np.arange(2, len(x) + 2)))
    def get_implict_matrix(rec_items, test_set):
        rel_matrix = [[0] * rec_items.shape[1] for _ in range(rec_items.shape[0])]
        for user in range(len(test_set)):
            for index, item in enumerate(rec_items[user]):
                if item in test_set[user]:
                    rel_matrix[user][index] = 1
        return np.array(rel_matrix)
    rel_matrix = get_implict_matrix(rec_items, test_set)
    ndcgs = []
    for user in range(len(test_set)):
        rels = rel_matrix[user]
        dcg = DCG(rels)
        idcg = DCG(sorted(rels, reverse=True))
        ndcg = dcg / idcg if idcg != 0 else 0
        ndcgs.append(ndcg)
    return ndcgs
 
 
# 假设 top-20 推荐,一共 5 个 user, 50 个 item ,隐式反馈数据集.
users = np.array([0])
# test_set 表示 5 个用户在测试集中分表交互过那些 item
test_set = [
    [0, 21, 31, 41, 49]
]
rec_items=np.array([
    [0,  9,  5,  6, 7, 50, 8, 31, 21, 1]
])
# model = Model(20)
# rec_items = model(users)
print("truth click", test_set)
print("rec_items", rec_items)
ndcgs = nDCG(rec_items, test_set)
print(ndcgs)
 
print('-'*10)
 
dcg=1/np.log(2)+1/np.log(9)+1/np.log(10)
idcg=1/np.log(2)+1/np.log(3)+1/np.log(4)
ndcg=(1/np.log(2)+1/np.log(9)+1/np.log(10))/(1/np.log(2)+1/np.log(3)+1/np.log(4))
print(dcg, idcg, ndcg)

In [ ]:
rank_chunk = np.array([
    [[0, 1, 2], [2, 3, 4]],  # 第一个样本的排名结果
    [[1, 2, 3], [3, 4, 0]]   # 第二个样本的排名结果
])

In [ ]:
n_samples, k, topk = rank_chunk.shape  # [batch, k, rank]
rank_chunk_reshape = np.reshape(rank_chunk, [-1, topk])
rank_chunk_reshape

In [ ]:
u_k_i = np.zeros([n_samples * k, 5])
u_k_i

In [ ]:
np.arange(len(u_k_i))

In [ ]:
for i in range(topk):
    u_k_i[np.arange(len(u_k_i)), rank_chunk_reshape[:, i]] = 1 / (i + 10)
u_k_i